In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt

from sklearn.preprocessing import StandardScaler

In [44]:
data = pd.read_csv("data.dat")
dataset = data.copy()
dataset.head(100)

,load_index,time,eps,eps_V,str_s,str_d,str,Wsto,Wdis,Wtotal,Esto,Edis,Etotal
0,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
1,0.0,1.0,-0.005,-0.000049,-0.000049,-0.004902,-0.004951,0.000025,2.402922e-07,0.000025,0.000025,2.402922e-07,0.000025
2,0.0,2.0,-0.010,-0.000146,-0.000146,-0.009708,-0.009854,0.000048,9.424147e-07,0.000049,0.000073,1.182707e-06,0.000074
3,0.0,3.0,-0.015,-0.000290,-0.000290,-0.014419,-0.014710,0.000071,2.079196e-06,0.000074,0.000144,3.261903e-06,0.000148
4,0.0,4.0,-0.020,-0.000481,-0.000481,-0.019039,-0.019519,0.000094,3.624699e-06,0.000098,0.000238,6.886602e-06,0.000245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,95.0,-0.025,-0.035366,-0.035366,0.045732,0.010366,0.000031,2.091395e-05,0.000052,0.001081,2.117821e-03,0.003199
96,0.0,96.0,-0.020,-0.034869,-0.034869,0.049737,0.014869,0.000050,2.473773e-05,0.000074,0.001131,2.142558e-03,0.003273
97,0.0,97.0,-0.015,-0.034332,-0.034332,0.053664,0.019332,0.000068,2.879799e-05,0.000097,0.001199,2.171356e-03,0.003370
98,0.0,98.0,-0.010,-0.033757,-0.033757,0.057513,0.023757,0.000086,3.307802e-05,0.000119,0.001284,2.204434e-03,0.003489


In [45]:
#Using the relevant variables for training
#eps = total strain at present
#eps_n1 = total strain at present timestep +1 (future)
#eps_V = viscous strain at present
#eps_V-n1 = viscous strain at future
#str_n1 = total stress at future

df = np.zeros((len(dataset)-1,5))
df[:,0] = dataset.iloc[:-1,2]
df[:,1] = dataset.iloc[1:,2]
df[:,2] = dataset.iloc[:-1,3]
df[:,3] = dataset.iloc[1:,3]
df[:,4] = dataset.iloc[1:,6]
df = pd.DataFrame(df, columns=["eps", "eps_n1", "eps_V", "eps_V_n1", "str_n1"])

In [52]:
print(f"Total rows in our database: {len(df)}")

Total rows in our database: 1478740


In [53]:
#We only select 10.000 rows to limit the trianing procedure (know your problem)
num_data = 10000
x = df.iloc[:num_data, :3] #input: present total strain, future total strain, and present viscous strain
y = df.iloc[:num_data, 3:] #predict: future viscous strain and future total stress

my_scaler = StandardScaler() #apply scaler
x = my_scaler.fit_transform(x)#x.to_numpy().reshape(-1,1)) #use the commented reshape if you want to train RNN 
y = my_scaler.fit_transform(y)#y.to_numpy().reshape(-1,1)) #use the commented reshape if you want to train RNN
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=888)

In [69]:
#Let us tune the hyperparameters our our deep learning model
#We arbitrarily choose 3 middle layers which number of neurons and activation functions are optimised
#We also optimise the learning rate then compile the model and later we will save the best hyperparameters
#This is for the feed forward neural network algorithm (FFNN)
def model_builder(hp):
    inputs = keras.Input(shape=(x_train.shape[1], 3))
    hp_units_1 = hp.Int("units_1", min_value=4, max_value=256, step=16)
    hp_units_2 = hp.Int("units_2", min_value=4, max_value=256, step=16)
    hp_units_3 = hp.Int("units_3", min_value=4, max_value=256, step=16)

    hp_activation_1 = hp.Choice("activation_1", values = ["tanh", "sigmoid", "relu"])
    hp_activation_2 = hp.Choice("activation_2", values = ["tanh", "sigmoid", "relu"])
    hp_activation_3 = hp.Choice("activation_3", values = ["tanh", "sigmoid", "relu"])
    v = layers.Dense(units=hp_units_1, activation=hp_activation_1, use_bias=True)(inputs)
    v = layers.Dense(units=hp_units_2, activation=hp_activation_2, use_bias=True)(v)
    v = layers.Dense(units=hp_units_3, activation=hp_activation_3, use_bias=True)(v)

    outputs = layers.Dense(2)(v)
    model = keras.Model(inputs=inputs, outputs=outputs, name="model_FFNN_tuning")
    
    hp_learning_rate = hp.Choice("learning_rate", values = [1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss="mse", metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

In [70]:
#Implementation of the hyperparameters optimisation
#Please note that this may take a while to run, depends on your computing power
import os
tuner = kt.Hyperband(model_builder,
                     objective = "mean_squared_error",
                     max_epochs = 50,
                     factor = 3,
                     directory = r"./my_dir",
                     project_name = "1D_visco_FFNN",
                     overwrite=True,
                    )
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train, validation_split=0.2, callbacks=[stop_early])

Trial 90 Complete [00h 00m 12s]
mean_squared_error: 0.01119103655219078

Best mean_squared_error So Far: 0.010301079601049423
Total elapsed time: 00h 05m 37s
INFO:tensorflow:Oracle triggered exit


In [71]:
#Printing out the optimised hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)
print(best_hps.get("units_1"))
print(best_hps.get("units_2"))
print(best_hps.get("units_3"))
print(best_hps.get("learning_rate"))

print(best_hps.get("activation_1"))
print(best_hps.get("activation_2"))
print(best_hps.get("activation_3"))

84
84
164
0.001
relu
relu
relu


In [86]:
#Create the layers
inputs = keras.Input(shape=(x_train.shape[1], 3))
x = layers.Dense(best_hps.get("units_1"), activation=best_hps.get("activation_1"))(inputs)
x = layers.Dense(best_hps.get("units_2"), activation=best_hps.get("activation_2"))(x)
x = layers.Dense(best_hps.get("units_3"), activation=best_hps.get("activation_3"))(x)
outputs = layers.Dense(2)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_hps.get("learning_rate")), 
                  loss="mse", metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=20, batch_size=20, verbose=0)
test_scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", test_scores[0])
print("Test Accuracy", test_scores[1])

Test Loss: 0.015282166190445423
Test Accuracy 0.9894999861717224


In [80]:
test_scores = model.evaluate(x_test, y_test, verbose=1)
print("Test Loss:", test_scores[0])
print("Test Accuracy", test_scores[1])

63/63 [==============================] - 0s 717us/step - loss: 0.0160 - accuracy: 0.9810
Test Loss: 0.016011305153369904
Test Accuracy 0.9810000061988831


In [ ]:
#Let us tune the hyperparameters our our deep learning model
#We arbitrarily choose 3 middle layers which number of neurons and activation functions are optimised
#We also optimise the learning rate then compile the model and later we will save the best hyperparameters
#This is for RNN (recurrent neural network) algorithm, we do not use, please modify the input file so that the dimensions match
#def model_builder(hp):
#    inputs = keras.Input(shape=(x_train.shape[1], 1))
#    hp_units_1 = hp.Int("units_1", min_value=4, max_value=256, step=16)
#    hp_units_2 = hp.Int("units_2", min_value=4, max_value=256, step=16)
#    hp_units_3 = hp.Int("units_3", min_value=4, max_value=256, step=16)
#
#    hp_activation_1 = hp.Choice("activation_1", values = ["tanh", "sigmoid", "relu"])
#    hp_activation_2 = hp.Choice("activation_2", values = ["tanh", "sigmoid", "relu"])
#    hp_activation_3 = hp.Choice("activation_3", values = ["tanh", "sigmoid", "relu"])
#    
#    v = layers.LSTM(units=hp_units_1, activation=hp_activation_1, return_sequences = True, use_bias=True)(inputs)
#    v = layers.LSTM(units=hp_units_2, activation=hp_activation_2, return_sequences = True, use_bias=True)(v)
#    v = layers.LSTM(units=hp_units_3, activation=hp_activation_3, return_sequences = True, use_bias=True)(v)
#    
#outputs = layers.TimeDistributed(layers.Dense(1))(v) We do not use RNN, however this can be changed
#    model = keras.Model(inputs=inputs, outputs=outputs, name="model_RNN_tuning")
#    
#    hp_learning_rate = hp.Choice("learning_rate", values = [1e-2, 1e-3, 1e-4])
#    
#    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
#                  loss="mse", metrics=[tf.keras.metrics.MeanSquaredError()])
#    return model

In [ ]:
inputs = keras.Input(shape=(x_train.shape[1], 1))
v = layers.LSTM(units = 32, return_sequences = True, activation="relu", use_bias=True)(inputs)
v = layers.LSTM(units = 32, return_sequences = True, activation="relu", use_bias=True)(v)
v = layers.LSTM(units = 32, return_sequences = True)(v)
outputs = layers.TimeDistributed(layers.Dense(1))(v)

model_RNN = keras.Model(inputs=inputs, outputs=outputs, name="model_RNN")
model_RNN.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                  loss="mse", metrics=[tf.keras.metrics.MeanSquaredError()])